# Exercise 4.9a

### Content List:
#### -- Importing libraries & datasets
#### -- Wrangle Data
#### -- Complete fundamental data quality & consistency checks
#### -- Merge customer data with rest of dataset
#### -- Export

## Importing libraries & datasets 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# define path variable
# change path to 'jsok' if using desktop!!
path = r'/Users/jsok/Instacart Basket Analysis'

In [3]:
# read 'orders_products_merged.pkl' and store in 'ords_prods_merge'
ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_merged.pkl'))

In [4]:
# read 'customers.csv' and store in 'cust'
cust = pd.read_csv(os.path.join(path,'02 Data','Original Data','customers.csv'))

## Wrangle data so that it follow consitent logic (change column names, data types, drop columns)

In [5]:
# view imported df
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


#### Column names were kept because gender, state, age, dependants, fam_status, and income could be further analyzed by department or sales.

In [6]:
# view imported df
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [7]:
# rename 'cust' column names
cust = cust.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'STATE':'state', 'Gender':'gender', 'Age':'age', 'n_dependants':'dependants'})

In [8]:
# change 'user_id' datatype to 'str'
cust['user_id'] = cust['user_id'].astype('str')

In [9]:
# descriptive statistics on 'cust' df
cust.describe()

,age,dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


## Complete the fundamental data quality and consistency checks (missing values, duplicates, or mixed types)

In [10]:
# locate any missing values
cust.isnull().sum()

user_id            0
first_name     11259
last_name          0
gender             0
state              0
age                0
date_joined        0
dependants         0
fam_status         0
income             0
dtype: int64

##### Missing 'first_name' values are okay, since no last_name values are missing.

In [11]:
# checking for MIXED data types
# returns column names if there exists a mixed type column

for col in cust.columns.tolist():
  weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (cust[weird]) > 0:
    print (col)

first_name


In [12]:
# change 'first_name' to 'str' since it was returned as a mixed data type!!
cust['first_name'] = cust['first_name'].astype('str')

In [13]:
# find duplicates
df_dups = cust[cust.duplicated()]
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,dependants,fam_status,income


In [14]:
# drop duplicates in 'cust' df
cust.drop_duplicates()

,user_id,first_name,last_name,gender,state,age,date_joined,dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...,...,...
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799


## Combining customer data with the rest of prepared Instacart data.


In [15]:
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   order_day_of_week       int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _merge                  category
 14  price_range_loc         object  
 15  Busiest_day             object  
 16  Busiest_days            object  
 17  Busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  avg_order_price         float64 
 21  spendi

In [16]:
# change data types for primary, foreign keys
# change 'order_id', 'user_id', 'product_id', 'aisle_id', 'department_id' datatype to 'str'
ords_prods_merge['order_id'] = ords_prods_merge['order_id'].astype('str')
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')
ords_prods_merge['product_id'] = ords_prods_merge['product_id'].astype('str')
ords_prods_merge['aisle_id'] = ords_prods_merge['aisle_id'].astype('str')
ords_prods_merge['department_id'] = ords_prods_merge['department_id'].astype('str')

#### combine 'cust' data with 'ords_prods_merge' data using 'user_id' (now the same data type!)

In [17]:
# merge data via INNER join on 'user_id'
ords_prods_all = ords_prods_merge.merge(cust, on = ['user_id'])

In [18]:
# view dimenions of merged df
ords_prods_all.shape

(32404859, 33)

## Export

In [19]:
# Export
ords_prods_all.to_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_all.pkl'))